In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[6],4
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000833                    1 -0.000915
0.000750        2 -0.000837                    2 -0.000942
0.001052        3 -0.000843                    3 -0.000918
0.001476        4 -0.000849                    4 -0.000928
0.002070        5 -0.000855                    5 -0.000935
0.002904        6 -0.000860                    6 -0.000944
0.004074        7 -0.000864                    7 -0.000951
0.005714        8 -0.000868                    8 -0.000958
0.008015        9 -0.000871                    9 -0.000964
0.011243       10 -0.000873                   10 -0.000972
0.015771       11 -0.000875                   11 -0.000986
0.022122       12 -0.000876                   12 -0.000996
0.031031       13 -0.000876                   13 -0.001006
0.043528       14 -0.000874                   14 -0.001018
0.061057       15 -0.000855                   15 -0.001040
0.085645       16 -0.000736                   16 -0.001028
0.120136       17 -0.000394                   17 -0.000891
0.168516       18  0.000280                   18 -0.000543
0.236378       19  0.001452                   19  0.000117
0.331549       20  0.003346                   20  0.001234
0.465100       21  0.006255                   21  0.003034
0.652400       22  0.010061                   22  0.005604
0.915100       23  0.011463                   23  0.006711
1.283650       24  0.009597                   24  0.005648
1.800600       25  0.006923                   25  0.004003
2.525700       26  0.004158                   26  0.002250
3.542800       27  0.002058                   27  0.000894
4.969550       28  0.000622                   28 -0.000046
6.970850       29 -0.000317                   29 -0.000674
9.778100       30 -0.000897                   30 -0.001069
13.715850      31 -0.001222                   31 -0.001290
19.239350      32 -0.001371                   32 -0.001385
26.987250      33 -0.001404                   33 -0.001381
37.855300      34 -0.001358                   34 -0.001290
53.100050      35 -0.001215                   35 -0.001163
73.887500      36 -0.000984                   36 -0.001006
97.662500      37 -0.000853                   37 -0.000904
121.437500     38 -0.000978                   38 -0.001017
145.212500     39 -0.001256                   39 -0.001243
168.987500     40 -0.001508                   40 -0.001433
192.762500     41 -0.001716                   41 -0.001625
216.537500     42 -0.001871                   42 -0.001823
240.312500     43 -0.001961                   43 -0.001951
264.087500     44 -0.001981                   44 -0.001990
287.862500     45 -0.001926                   45 -0.001950
311.637500     46 -0.001792                   46 -0.001835
335.412500     47 -0.001575                   47 -0.001621
359.187500     48 -0.001272                   48 -0.001336
382.962500     49 -0.000883                   49 -0.000974
406.737500     50 -0.000405                   50 -0.000520
430.512500     51  0.000162                   51  0.000022
454.287500     52  0.000819                   52  0.000649
478.062500     53  0.001566                   53  0.001371
501.837500     54  0.002404                   54  0.002189
525.612500     55  0.003332                   55  0.003078
549.387500     56  0.004352                   56  0.004059
573.162500     57  0.005462                   57  0.005139
596.937500     58  0.006662                   58  0.006320
620.712500     59  0.007949                   59  0.007601
644.487500     60  0.009314                   60  0.008940
668.262500     61  0.010732                   61  0.010316
692.037500     62  0.012141                   62  0.011704
715.812500     63  0.013455                   63  0.013026
739.587500     64  0.014629                   64  0.014250
763.362500     65  0.015720                   65  0.015416
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -66.833204  0.000000e+00 -66.833204 -66.878102   
0.000624    2     -66.833204  3.228703e-08 -66.833204 -66.878102   
0.000876    3     -66.833204  4.556105e-08 -66.833204 -66.878102   
0.001229    4     -66.833205  6.497257e-08 -66.833204 -66.878102   
0.001723    5     -66.833205  9.338455e-08 -66.833205 -66.878103   
0.002417    6     -66.833205  1.349522e-07 -66.833205 -66.878103   
0.003391    7     -66.833205  1.957649e-07 -66.833205 -66.878103   
0.004757    8     -66.833205  2.847215e-07 -66.833205 -66.878103   
0.006672    9     -66.833205  4.148220e-07 -66.833205 -66.878103   
0.009359    10    -66.833206  6.050726e-07 -66.833205 -66.878104   
0.013128    11    -66.833206  8.832401e-07 -66.833206 -66.878104   
0.018415    12    -66.833207  1.289901e-06 -66.833206 -66.878105   
0.025830    13    -66.833209  1.884328e-06 -66.833207 -66.878107   
0.036232    14    -66.833211  2.753232e-06 -66.833209 -66.878109   
0.050823    15    -66.833214  4.037568e-06 -66.833210 -66.878111   
0.071291    16    -66.833218  6.198159e-06 -66.833212 -66.878116   
0.100000    17    -66.833225  1.050992e-05 -66.833215 -66.878122   
0.140271    18    -66.833236  1.953263e-05 -66.833216 -66.878133   
0.196760    19    -66.833252  3.819481e-05 -66.833215 -66.878150   
0.275997    20    -66.833277  7.632679e-05 -66.833201 -66.878177   
0.387100    21    -66.833310  1.531675e-04 -66.833157 -66.878215   
0.543100    22    -66.833347  3.060117e-04 -66.833041 -66.878267   
0.761700    23    -66.833375  5.952347e-04 -66.832781 -66.878334   
1.068500    24    -66.833399  1.035201e-03 -66.832364 -66.878422   
1.498800    25    -66.833451  1.576008e-03 -66.831875 -66.878558   
2.102400    26    -66.833563  2.183792e-03 -66.831380 -66.878766   
2.949000    27    -66.833777  2.813719e-03 -66.830963 -66.879078   
4.136600    28    -66.834123  3.449998e-03 -66.830674 -66.879521   
5.802500    29    -66.834641  4.090618e-03 -66.830550 -66.880128   
8.139200    30    -66.835375  4.736497e-03 -66.830638 -66.880938   
11.417000   31    -66.836378  5.391343e-03 -66.830986 -66.881999   
16.014700   32    -66.837712  6.060375e-03 -66.831652 -66.883365   
22.464000   33    -66.839448  6.748885e-03 -66.832700 -66.885101   
31.510500   34    -66.841664  7.459823e-03 -66.834203 -66.887265   
44.200100   35    -66.844431  8.185641e-03 -66.836245 -66.889876   
62.000000   36    -66.847634  8.827664e-03 -66.838807 -66.892936   
85.775000   37    -66.850872  9.293765e-03 -66.841578 -66.896251   
109.550000  38    -66.853605  9.623254e-03 -66.843981 -66.899152   
133.325000  39    -66.856819  1.008267e-02 -66.846735 -66.902485   
157.100000  40    -66.861210  1.093766e-02 -66.850273 -66.906779   
180.875000  41    -66.866858  1.233889e-02 -66.854519 -66.912036   
204.650000  42    -66.873796  1.444373e-02 -66.859352 -66.918417   
228.425000  43    -66.882036  1.741507e-02 -66.864621 -66.926147   
252.200000  44    -66.891560  2.141658e-02 -66.870144 -66.935185   
275.975000  45    -66.902338  2.661421e-02 -66.875723 -66.945411   
299.750000  46    -66.914323  3.317400e-02 -66.881148 -66.956742   
323.525000  47    -66.927455  4.126081e-02 -66.886194 -66.969143   
347.300000  48    -66.941666  5.103746e-02 -66.890629 -66.982396   
371.075000  49    -66.956877  6.266409e-02 -66.894212 -66.996497   
394.850000  50    -66.972995  7.629529e-02 -66.896699 -67.011447   
418.625000  51    -66.989920  9.207958e-02 -66.897840 -67.027107   
442.400000  52    -67.007549  1.101653e-01 -66.897384 -67.043289   
466.175000  53    -67.025775  1.306971e-01 -66.895077 -67.059975   
489.950000  54    -67.044482  1.538154e-01 -66.890667 -67.077111   
513.725000  55    -67.063555  1.796569e-01 -66.883898 -67.094623   
537.500000  56    -67.082867  2.083541e-01 -66.874513 -67.112214   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')